# Setup and Definitions

TODO blurb

In [1]:
import sympy as sp
from sympy import sin, cos, tan
from sympy.abc import alpha, psi, sigma, i, d, iota, delta
from analemma.algebra import frame, render, util

Will also want $s$ on the $n$ frame TODO

## Fixed Stars Frame

First, we define the geometric algebra of 3-space and some basis blades from the frame of the "fixed stars"

![Earth's orientation and orbit](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/MainArena.png?token=GHSAT0AAAAAAB73Q3JM6JGFDMRHPPAJKRQ2ZAWFO7Q "Earth's orientation and orbit.").

In [2]:
e1, e2, e3 = frame.base("e")

def check_orthnormal(three_frame) -> None:
    """
    Check that a frame of three vectors is orthonormal

    Ensure that the geometric product of the vectors is equal to the pseudoscalar
    """
    v1, v2, v3 = three_frame
    # this is more elegant
    # assert (v1*v2*v3).trigsimp().obj.equals((e1^e2^e3).obj)
    # but this is significantly faster and equivalent
    assert v1|v1 == v2|v2 == v3|v3 == frame.scalar_element(1)
    assert v1|v2 == v1|v3 == v2|v3 == frame.scalar_element(0)

check_orthnormal((e1, e2, e3))

### Earth Frame

Now, let the tilt of the earth's plane (axis) of rotation be $\alpha$ and measure the earth's rotation by $\psi$. Then we can define the Earth frame as follows.

In [3]:
f1, f2, f3 = frame.planet()
check_orthnormal((f1, f2, f3))
render.frame("f", (f1,f2,f3))

<IPython.core.display.Math object>

The equatorial plane should only depend on the tilt of the Earth's axis of spin $\alpha$, not the angle by which it has rotated relative to the fixed stars $\psi$.

In [4]:
render.expression(r"f_1 \wedge f_2", (f1^f2))

<IPython.core.display.Math object>

### Surface Frame

Define an orthonormal frame embedded in the Earth's surface, with $n_1$ pointing South, $n_2$ pointing East and $n_3$ pointing up.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.png?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface.").

<!-- ![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.svg?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface."). -->

In [5]:
n1, n2, n3 = frame.surface()

check_orthnormal((n1, n2, n3))

render.frame("n", (n1,n2,n3))

<IPython.core.display.Math object>

### Orbit Rotor and Meridian Plane

Earth orbit rotor $R_\sigma$, and vector parallel to rays of sunshine, $s$.

In [6]:
s = frame.sunray()

render.expression("s", s)

<IPython.core.display.Math object>

The meridian plane, $M = n_1 \wedge n_3$.

In [7]:
M = frame.meridian_plane()
render.expression("M", M)

<IPython.core.display.Math object>

The noon line is the intersection of the sunshine vector $s$ and the meridian plane $M$, which occurs where $s \wedge M$ vanishes.

In [8]:
render.multivector( (s^M).trigsimp() )

<IPython.core.display.Math object>

In [9]:
coeff = (s^M).trigsimp().get_coefs(3)[0]
soln = sp.solve(coeff.subs(sin(psi), tan(psi)*cos(psi)), tan(psi))[0]

assert soln.equals(tan(sigma)/cos(alpha))
render.expression( r"\tan(\psi)", soln )

<IPython.core.display.Math object>

### Dial face and gnomon

Define an orthonormal frame $u_1, u_2, u_3$ as the unevaluated version of $n_1, n_2, n_3$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/DialFrame.png?token=GHSAT0AAAAAAB73Q3JNJN46TIEHP3QCWWGYZAWGADA "Frame embedded in the sundial's face.").


Frame embedded in dial face.

In [10]:
m1, m2, m3 = frame.dial()

render.frame("m", (m1,m2,m3))

<IPython.core.display.Math object>

Dial face expressed relative to $n$ basis: $G_n$.

In [11]:
Gn = (m1^m2).trigsimp()

# check that we get the same result by rotating u1^u2
nn1, nn2, nn3 = frame.base("n")
assert Gn == util.rotate(util.rotate(nn1^nn2, i, nn1^nn3), d, nn1^nn2).trigsimp()

render.expression( "G_n", Gn)


<IPython.core.display.Math object>

The gnomon expressed relative to the $u$ frame, $g_u$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/Gnomon.png?token=GHSAT0AAAAAAB73Q3JNHJMDP6T55SWPTQFGZAWGBNA "The gnomon.").


In [12]:
gn = frame.gnomon("n", zero_decl=False)

# extra manipulation to display exactly as in paper
render.expression("g_n", sp.collect(sp.trigsimp(gn.obj), -sin(iota)))

<IPython.core.display.Math object>

Projected onto the fixed-stars basis, the gnomon is

In [13]:
g = util.project_vector(gn, target_frame=(nn1, nn2, nn3), render_frame=(n1, n2, n3))
g = g.trigsimp()

render.align("g", g)

<IPython.core.display.Math object>

The gnomon lies in the meridian plane when the following trivector vanishes:

In [14]:
M_wedge_g = M^g
assert M_wedge_g.obj.trigsimp().equals((sin(delta)*sin(iota)*e1^e2^e3).obj)
render.multivector( M_wedge_g )

<IPython.core.display.Math object>

Will also want $s$ on the $n$ frame TODO but not using it here

In [15]:
sn = util.project_vector(s, target_frame=(n1, n2, n3), render_frame=(nn1, nn2, nn3))

render.align("s_n", sn)


<IPython.core.display.Math object>